In [1]:
import numpy as np

In [2]:
# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError

    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

# Mon réseau de Neurones from scratch en POO 

In [3]:
# une classe qui hérite la classe base (Layer) et permet de créer un fully Connected layer

class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    # returns output for a given input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

 ##  Une classe qui hérite de la classe layer et qui permet l'activation de chaque layer:

Nous avons deux modes d'activations des layers: une pour la forward et une pour la backward.


In [4]:
# Classe qui hérite la classe base et permet de propager les activations de la forward et la backward 

class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [5]:
# activation function and its derivative

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

def sigmoid(x):
    z = np.exp(-x)
    return 1 / (1+ z)

def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))

In [6]:
# loss function and its derivative

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred-y_true)/y_true.size

## La classe **Network** 

Permet de construire notre réseau et l'entrainer

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

## Entrainer les données XOR 

In [8]:

#training data

x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])
# print(x_train)
# print(y_train)

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=500, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/500   error=0.726282
epoch 2/500   error=0.352778
epoch 3/500   error=0.313626
epoch 4/500   error=0.306807
epoch 5/500   error=0.304635
epoch 6/500   error=0.303588
epoch 7/500   error=0.302922
epoch 8/500   error=0.302421
epoch 9/500   error=0.302003
epoch 10/500   error=0.301636
epoch 11/500   error=0.301300
epoch 12/500   error=0.300988
epoch 13/500   error=0.300692
epoch 14/500   error=0.300410
epoch 15/500   error=0.300138
epoch 16/500   error=0.299873
epoch 17/500   error=0.299616
epoch 18/500   error=0.299363
epoch 19/500   error=0.299116
epoch 20/500   error=0.298871
epoch 21/500   error=0.298630
epoch 22/500   error=0.298392
epoch 23/500   error=0.298155
epoch 24/500   error=0.297921
epoch 25/500   error=0.297688
epoch 26/500   error=0.297457
epoch 27/500   error=0.297227
epoch 28/500   error=0.296999
epoch 29/500   error=0.296772
epoch 30/500   error=0.296546
epoch 31/500   error=0.296320
epoch 32/500   error=0.296096
epoch 33/500   error=0.295873
epoch 34/500   erro

epoch 305/500   error=0.002089
epoch 306/500   error=0.002054
epoch 307/500   error=0.002020
epoch 308/500   error=0.001987
epoch 309/500   error=0.001955
epoch 310/500   error=0.001924
epoch 311/500   error=0.001894
epoch 312/500   error=0.001864
epoch 313/500   error=0.001836
epoch 314/500   error=0.001808
epoch 315/500   error=0.001781
epoch 316/500   error=0.001755
epoch 317/500   error=0.001730
epoch 318/500   error=0.001705
epoch 319/500   error=0.001681
epoch 320/500   error=0.001658
epoch 321/500   error=0.001635
epoch 322/500   error=0.001613
epoch 323/500   error=0.001591
epoch 324/500   error=0.001570
epoch 325/500   error=0.001549
epoch 326/500   error=0.001529
epoch 327/500   error=0.001510
epoch 328/500   error=0.001490
epoch 329/500   error=0.001472
epoch 330/500   error=0.001454
epoch 331/500   error=0.001436
epoch 332/500   error=0.001418
epoch 333/500   error=0.001401
epoch 334/500   error=0.001385
epoch 335/500   error=0.001368
epoch 336/500   error=0.001353
epoch 33